In [102]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.manifold import TSNE
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

%matplotlib inline 

In [103]:
data= pd.read_csv("subset_data.csv")

In [104]:
large_data = pd.read_csv("data.csv", sep="\t")
#data = pd.read_csv("data.csv", sep="\t")

In [144]:
data.dropna(inplace=True)
# data.reset_index(inplace=True)
large_data.dropna(inplace = True)
# large_data.reset_index(inplace=True)

In [6]:
data

,Unnamed: 0,date_time,site_name,user_location_country,user_location_region,user_location_city,user_location_latitude,user_location_longitude,orig_destination_distance,user_id,...,srch_destination_id,hotel_country,is_booking,hotel_id,prop_is_branded,prop_starrating,distance_band,hist_price_band,popularity_band,cnt
0,1,2015-09-21 22:47:00,EXPEDIA.COM,UNITED STATES OF AMERICA,CA,BRENTWOOD,37.923810,-121.696220,5873.0280,-2147479371,...,187569808,FRANCE,0,133800525,1,4.0,C,M,VH,2
1,2,2015-11-15 21:15:00,EXPEDIA.COM,UNITED STATES OF AMERICA,CA,BRENTWOOD,37.923810,-121.696220,5329.1407,-2147479371,...,5526772,UNITED KINGDOM,0,439889,1,4.0,M,H,VH,1
2,3,2015-11-15 21:26:00,EXPEDIA.COM,UNITED STATES OF AMERICA,CA,BRENTWOOD,37.923810,-121.696220,5329.1407,-2147479371,...,5526772,UNITED KINGDOM,0,41796865,0,4.0,M,M,VH,1
3,4,2015-11-15 23:06:00,EXPEDIA.COM,UNITED STATES OF AMERICA,CA,BRENTWOOD,37.923810,-121.696220,5329.1407,-2147479371,...,5526772,UNITED KINGDOM,0,363054980,0,4.0,M,H,M,1
4,5,2015-05-30 23:57:00,EXPEDIA.COM,UNITED STATES OF AMERICA,SD,SIOUX FALLS,43.527260,-96.773615,161.9662,-2147476571,...,82893,UNITED STATES OF AMERICA,0,127643119,1,3.0,VC,VH,M,1
5,6,2015-11-27 20:57:00,EXPEDIA.COM,UNITED STATES OF AMERICA,SD,SIOUX FALLS,43.527260,-96.773615,195.1040,-2147476571,...,187860278,UNITED STATES OF AMERICA,0,24451838,1,3.0,M,M,M,3
6,7,2015-11-27 21:03:00,EXPEDIA.COM,UNITED STATES OF AMERICA,SD,SIOUX FALLS,43.527260,-96.773615,195.1040,-2147476571,...,187860278,UNITED STATES OF AMERICA,0,312727,1,4.0,M,L,H,1
7,8,2015-11-27 21:14:00,EXPEDIA.COM,UNITED STATES OF AMERICA,SD,SIOUX FALLS,43.527260,-96.773615,199.6104,-2147476571,...,5527020,UNITED STATES OF AMERICA,0,1986541,1,3.0,C,M,VH,4
8,9,2015-04-23 23:39:00,EXPEDIA.COM,UNITED STATES OF AMERICA,CA,LOS ANGELES,34.059768,-118.312427,5477.8567,-2147471748,...,5577023,JAPAN,0,343076,0,5.0,VC,VH,H,1
9,10,2015-05-27 11:31:00,EXPEDIA.CA,CANADA,BC,SURREY,49.151573,-122.795127,2790.0344,-2147412448,...,5526989,UNITED STATES OF AMERICA,0,16702706,0,3.0,C,L,H,1


In [106]:
def relabel(col, top_names):
    l = []
    for name in col:
        if name in top_names:
            l.append(name)
        else:
            l.append('other')
    return l

In [145]:
def oneHotEncode(col, n):
    # First find top 10 values in col and set everything else to 'other'
    top_names = col.value_counts()[:n].index
    convert_col = relabel(col, top_names)
    return pd.get_dummies(convert_col, drop_first=False)
def create_matrix(data, numerical_cols, one_hot_cols):
    df = pd.DataFrame()
    for col in numerical_cols:
        df = pd.concat([df, col], axis=1)
    for col in one_hot_cols:
#          df = pd.concat([df, oneHotEncode(col[0], col[1])], axis = 1)
        df = np.hstack((df, oneHotEncode(col[0], col[1])))
    return df

In [148]:
# category = [(data.user_location_region, 60), (data.user_location_city, 50), (data.site_name, 3), (data.popularity_band, 5)]
# m = create_matrix(data, [data.is_mobile, data.srch_adults_cnt, data.srch_children_cnt, data.prop_starrating, data.prop_is_branded], category)
# print(m.shape)
category = [(data.user_location_region, 60), (data.user_location_city, 50), (data.site_name, 3)]
m = create_matrix(data, [data.is_mobile, data.srch_adults_cnt, data.srch_children_cnt, data.prop_starrating, data.prop_is_branded], category)
print(m.shape)

(78977, 120)


In [149]:
m.dropna(inplace=True)
print(m.shape)

AttributeError: 'numpy.ndarray' object has no attribute 'dropna'

In [150]:
clf = RandomForestClassifier(n_estimators=64)
Y = data.is_booking

In [151]:
Test_data = large_data[100001:101000]
print(Test_data)

         index            date_time    site_name     user_location_country  \
100001  126237  2015-07-02 11:09:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100002  126238  2015-09-15 16:25:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100003  126239  2015-09-15 19:39:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100004  126240  2015-10-08 16:44:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100005  126241  2015-10-08 16:55:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100006  126242  2015-10-09 21:09:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100007  126244  2015-11-18 20:53:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100008  126245  2015-11-18 20:53:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100009  126246  2015-11-18 20:53:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100010  126247  2015-11-18 20:53:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100011  126251  2015-11-30 07:46:00  EXPEDIA.COM  UNITED STATES OF AMERICA   
100012  126253  2015-05-15 15:30:00   EXPEDIA.CA                

In [152]:
Test_category = [(Test_data.user_location_region, 60), (Test_data.user_location_city, 50), (Test_data.site_name, 3), (data.popularity_band, 5)]
print(Test_category)

# Test_category = [(Test_data.user_location_region, 60), (Test_data.user_location_city, 50), (Test_data.site_name, 3)]
# print(Test_category)
# print(len(Test_category))

[(100001    CA
100002    VA
100003    VA
100004    TX
100005    TX
100006    TX
100007    IL
100008    IL
100009    IL
100010    IL
100011    TX
100012    ON
100013    ON
100014    CA
100015    WA
100016    WA
100017    CA
100018    CA
100019    MA
100020    OK
100021    OK
100022    OK
100023    NJ
100024    NJ
100025    NJ
100026    RI
100027    CA
100028    TX
100029    SC
100030    CA
          ..
100970    NW
100971    NW
100972    NW
100973    ON
100974    FL
100975    NW
100976    PA
100977    BW
100978    BW
100979    BW
100980    ME
100981    ME
100982    FL
100983    FL
100984    FL
100985    OH
100986    NY
100987    OH
100988    ON
100989    NY
100990    NY
100991    NC
100992    NS
100993    NS
100994    GA
100995    SK
100996    SK
100997    SK
100998    FL
100999    FL
Name: user_location_region, dtype: object, 60), (100001                 PITTSBURG
100002                CHESAPEAKE
100003                CHESAPEAKE
100004               SAN ANTONIO
100005               SAN

In [153]:
Test_m = create_matrix(Test_data, [Test_data.is_mobile, Test_data.srch_adults_cnt, Test_data.srch_children_cnt], Test_category)
# print(Test_m.shape)

# Test_m = create_matrix(Test_data, [Test_data.is_mobile, Test_data.srch_adults_cnt, Test_data.srch_children_cnt, Test_data.prop_starrating, Test_data.prop_is_branded], Test_category)
# print(Test_m.shape)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [142]:
print(Test_m)

        is_mobile  srch_adults_cnt  srch_children_cnt   AB   AL   AR   AZ  \
0             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
1             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
2             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
3             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
4             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
5             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
6             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
7             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
8             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
9             NaN              NaN                NaN  0.0  0.0  0.0  0.0   
10            NaN              NaN                NaN  0.0  0.0  0.0  0.0   
11            NaN              NaN                NaN  0.0  0.0  0.0  0.0   

In [154]:
Test_m.dropna(inplace=True)
print(Test_m.shape)

(0, 123)


In [155]:
clf.fit(m,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=64, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [55]:
predicted_results = clf.predict(Test_m)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [156]:
predicted_results
print("Predicted num: " + str(sum(predicted_results)))
print("Actual num: " + str(sum(Test_data.is_booking)))

Predicted num: 7
Actual num: 103
